In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
merged = pd.read_csv("D:/1 GSU Analytics/output1.csv")

In [4]:
pd.set_option('max_columns', 100)

In [39]:
unique_address = merged.drop_duplicates('Risk_Location_Address_1')[['Policy_Number','Risk_Location_Address_1','Risk_Location_City_Name','Named_Insured_State_Code']]
unique_address.head(5)

,Policy_Number,Risk_Location_Address_1,Risk_Location_City_Name,Named_Insured_State_Code
0,1000373652161,8786 Parsons Blvd,Jamaica,NY
9,1000373651161,16923 Sierra Lakes Pkwy,Fontana,CA
21,1000373650161,3827 E Sunset Rd,Las Vegas,NV
32,1000373649161,9469 W Atlantic Blvd,Coral Springs,FL
41,1000373648161,2643 Windmill Pkwy,Henderson,NV


In [18]:
unique_address.count()

index                       32912
Risk_Location_Address_1     32912
Risk_Location_City_Name     32912
Named_Insured_State_Code    32912
search_url                  32912
dtype: int64

In [41]:
unique_address['search_url'] = "www.google.com/search?q=propertyshark+" + unique_address["Risk_Location_Address_1"].str.replace(" ","+")+"+"+unique_address["Risk_Location_City_Name"].str.replace(" ","+")+"+"+unique_address['Named_Insured_State_Code']
unique_address = unique_address.reset_index()


In [42]:
unique_address.count()
unique_address.search_url[29]

'www.google.com/search?q=propertyshark+2382+Brooklyn+Queens+Expy+W+Astoria+NY'

In [54]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def hit_page(only_url,p_id):
    r  = requests.get("http://" +only_url)

    data = r.text

    soup = BeautifulSoup(data)

    tables = soup.findChildren('table', attrs={'class':'data'})
    total_tables = len(tables)
    
    # This will get the first (and only) table. Your page may have more.
    my_table = tables[total_tables - 2]
    # You can find children with multiple tags by passing a list of strings
    rows = my_table.findChildren(['th', 'td'])

    for row in rows:
        thdata = row.findChildren('th')
        cells = row.findChildren('td',attrs={'class':'r_align'})
        for cell in cells:
            value = cell.string
            print thdata[0].string
            print value

In [55]:
url = unique_address.search_url[20]
p_id = unique_address.Policy_Number[20]
r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data)
i = 1

for link in soup.findAll('h3', attrs={'class':'r'}):    
    if(i == 1):
        url = link.find('a')['href']
        #print(link.find('a')['href'])
    i= i +1   


only_url = find_between( url, "http://", "/&sa" )  

if("www.propertyshark.com/mason/Property" in only_url):
    print only_url
    hit_page(only_url,p_id)

www.propertyshark.com/mason/Property/49506/741-Hunts-Point-Ave-Bronx-NY-10474
Tax year
2015/2016
Current tax bill
$72,455
Projected tax bill
$77,325


In [53]:
dict1 = {'d1':{'a':1,'b':2}}
dict2 = {'d2':{'c':3,'d':4}}
dict1.update(dict2)
len(dict1)

2